[![Kaggle link](https://img.shields.io/badge/изначальный%20датасет-Открыть-orange?style=social&logo=kaggle)](https://www.kaggle.com/datasets/leomartinelli/bullying-in-schools)


# §1. Зависимости

In [252]:
import pandas as pd
from tabulate import tabulate
import numpy as np
%matplotlib inline 

## §2. Описание изначального датасета

In [253]:
data.shape

(56981, 18)

In [254]:
df_csv_path = 'data/Bullying_2018_RAW.csv'
bullying_df = pd.read_csv(df_csv_path) 

In [255]:


bull_desc = [
    ['Номер ответа','-'],
    ['В школе (за последние 12 месяцев)','Категориальный'], # - №2
    ['Вне школы (за последние 12 месяцев)','Категориальный'], # - №3
    ['Кибербуллинг (последние 12 месяцев?)','Категориальный'], # - №3
    ['Возраст респондента','Числовой'], # - №4
    ['Пол респондента','Категориальный'], # - №5
    ['Количество физических атак','Числовой'], # - №6
    ['Количество драк','Числовой'], # # - №7
    ['Ощущение одинокости','Категориальный'], # # - №8
    ['Количество близких друзей','Числовой'], # - №9
    ['Количество дней пропусков школы без разрешения','Числовой '], # - №10
    ['Доброта и отзывчивость окружающих','Категориальный'], # - №11
    ['Частота проблем взаимопонимания с родителями','Категориальный'], # - №12
    ['Ощущение одинокости большую части (частота)','Категориальный'], # # - №13
    ['Пропускали ли вы школу без разрешения (да/нет)','Категориальный'], # - №14
    ['Масса вашего тела в школе была ниже нормы?','Категориальный'], # - №15
    ['Масса вашего тела в школе была выше нормы?','Категориальный'], # - №16
    ['Вы страдали ожирением?','Категориальный'], # - №17
]


# §3. Статистика по датасету

In [262]:

def stats(data=bullying_df, desc=bull_desc, headers=('Номер','Название','Описание','Кат.','Числ.','Кол-во вариаций')):
    headers=('Номер','dtype','Описание','Тип','Кол-во пропусков','Кол-во вариаций')
    table = np.arange(data.shape[1]*len(headers))
    table = table.reshape(len(headers),data.shape[1])

    table = np.array([
                [x for x in range(1,19)],
                [d for d in data.dtypes],
                [a[0] for a in desc],
                [a[1] for a in desc],
                [data[col].isna().sum() for col in data.columns],
                [data[col].nunique() for col in data.columns]])

    table = table.transpose()

    print(
        tabulate(
            [data.shape],
            headers=('Кол-во строк', 'Кол-во столбцов'),
            tablefmt="github"),
        end='\n\n\n'
        )
    
    print(
        tabulate(
            table,
            headers=headers,
            tablefmt='github'
        ),
        end='\n\n\n'
        )

stats()

|   Кол-во строк |   Кол-во столбцов |
|----------------|-------------------|
|          56981 |                18 |


|   Номер | dtype   | Описание                                       | Тип            |   Кол-во пропусков |   Кол-во вариаций |
|---------|---------|------------------------------------------------|----------------|--------------------|-------------------|
|       1 | int64   | Номер ответа                                   | -              |                  0 |             56981 |
|       2 | object  | В школе (за последние 12 месяцев)              | Категориальный |                  0 |                 3 |
|       3 | object  | Вне школы (за последние 12 месяцев)            | Категориальный |                  0 |                 3 |
|       4 | object  | Кибербуллинг (последние 12 месяцев?)           | Категориальный |                  0 |                 3 |
|       5 | object  | Возраст респондента                            | Числовой       |                  0 

## Нормализация данных

In [257]:
def normalize_source(df=data):
    return df

normalize_source()


,record,Bullied_on_school_property_in_past_12_months,Bullied_not_on_school_property_in_past_12_months,Cyber_bullied_in_past_12_months,Custom_Age,Sex,Physically_attacked,Physical_fighting,Felt_lonely,Close_friends,Miss_school_no_permission,Other_students_kind_and_helpful,Parents_understand_problems,Most_of_the_time_or_always_felt_lonely,Missed_classes_or_school_without_permission,Were_underweight,Were_overweight,Were_obese
0,1,Yes,Yes,,13 years old,Female,0 times,0 times,Always,2,10 or more days,Never,Always,Yes,Yes,,,
1,2,No,No,No,13 years old,Female,0 times,0 times,Never,3 or more,0 days,Sometimes,Always,No,No,,,
2,3,No,No,No,14 years old,Male,0 times,0 times,Never,3 or more,0 days,Sometimes,Always,No,No,No,No,No
3,4,No,No,No,16 years old,Male,0 times,2 or 3 times,Never,3 or more,0 days,Sometimes,,No,No,No,No,No
4,5,No,No,No,13 years old,Female,0 times,0 times,Rarely,3 or more,0 days,Most of the time,Most of the time,No,No,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56976,57091,No,Yes,No,16 years old,Male,0 times,4 or 5 times,Sometimes,3 or more,0 days,Sometimes,Sometimes,No,No,No,Yes,No
56977,57092,No,No,No,16 years old,Male,0 times,0 times,Rarely,1,0 days,Sometimes,Never,No,No,,,
56978,57093,No,No,No,16 years old,Male,0 times,0 times,Sometimes,3 or more,0 days,Rarely,Sometimes,No,No,,,
56979,57094,No,Yes,No,16 years old,Female,0 times,0 times,Sometimes,2,0 days,Most of the time,Rarely,No,No,,,


In [258]:

print(
    tabulate(
    
            range(1,19),
            data.columns,
            [a[0] for a in column_desc],
            [a[1] for a in column_desc],
            [data[col].nunique() for col in data.columns],
        headers=('Номер','Название','Описание','Тип','Кол-во вариаций')
    )
)

TypeError: tabulate() got multiple values for argument 'headers'